In [1]:
import sounddevice as sd
import matplotlib.pyplot as plt
import numpy as np
import json
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from scipy.interpolate import interp1d

In [2]:
data_path = '.\\outside_data\\'
rec_file = 'rec_file.txt'

fs = 44100 // 2
duration = 1 
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
print("volume.GetVolumeRange(): (%s, %s, %s)" % volume.GetVolumeRange())

start_range, stop_range, step = volume.GetVolumeRange()
psoc_start = 48 # la psoc e mai mare rangeul dar poate merge asa
psoc_stop = 127
map_to_range = interp1d([start_range, stop_range],[psoc_start,psoc_stop]) # mapping from device range to psoc range

volume.GetVolumeRange(): (-65.25, 0.0, 0.03125)


In [3]:
curr_db = 0
id_sound = 0
id_data = 0
usr_input = ''
f = open(rec_file, 'w')

# should delete old outside_data

while usr_input != 'stop':
    usr_input = input('Is the volume good?[y \ n][type \'stop\' to exit]')
    if usr_input == 'y':
        pref_volume = volume.GetMasterVolumeLevel()
        # get decibels/volume and data from mic and save current
        # for intre current si pref - oleaca petru mai multe date
        print(curr_db, pref_volume)
        #for curr_db_iterator in range(curr_db, pref_volume):
        str_to_save = '{' + f'\"id\": {id_data}, \"current_vol\": {map_to_range(curr_db)}, \"outside_sound\": \"sound_{id_sound - 1}.npy\", \"pref_vol\": {map_to_range(pref_volume)}' + '}'
        f.write(str_to_save + '\n')
        id_data += 1
    elif usr_input == 'n':
        curr_db = volume.GetMasterVolumeLevel()
        # toti db/volume intre current si preffered poti fi pusi pentru a augmenta datele(mai sus)
        # rec sound from mic and save name and np file in outside_data
        data = sd.rec(duration * fs, samplerate=fs, channels=1)
        sd.wait()
        fn = f'sound_{id_sound}'
        id_sound += 1
        np.save(data_path + str(fn), data)
        print('please adjust volume!')
f.close()

please adjust volume!
-25.147287368774414 -30.77667808532715
-25.147287368774414 -30.77667808532715
please adjust volume!
-30.77667808532715 -14.372318267822266
-30.77667808532715 -14.372318267822266
please adjust volume!
-16.000192642211914 -26.80240821838379


# Data Engineering time

Get data from unwilling subjects and then I can:
- from curr_db to lowest volume are all gonna have the same pref_volume with the same background noise
- add slight noise to background noise
- displace pref_volume and background noise with same amount of db somehow maybe idk

In [4]:
# reading data
data_path = '.\\outside_data\\'
rec_file = 'rec_file.txt'
data_array = []
with open(rec_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        obj = json.loads(line)
        data_array.append(obj)

print(data_array)

[{'id': 0, 'current_vol': 96.55347582937657, 'outside_sound': 'sound_0.npy', 'pref_vol': 89.73781503845449}, {'id': 1, 'current_vol': 96.55347582937657, 'outside_sound': 'sound_0.npy', 'pref_vol': 89.73781503845449}, {'id': 2, 'current_vol': 89.73781503845449, 'outside_sound': 'sound_1.npy', 'pref_vol': 109.59903228876692}, {'id': 3, 'current_vol': 89.73781503845449, 'outside_sound': 'sound_1.npy', 'pref_vol': 109.59903228876692}, {'id': 4, 'current_vol': 107.62811925310741, 'outside_sound': 'sound_2.npy', 'pref_vol': 94.54957472410238}]


In [5]:
aug_data = []
last_id = data_array[-1]['id']
# same preference for all dBs until curr_db
for data_obj in data_array:
    curr_db = data_obj['current_vol']
    print(data_obj, curr_db)
    while curr_db > psoc_start:
         curr_db -= 0.1
         data_new = {}
         data_new['id'] = last_id + 1
         data_new['current_vol'] = curr_db
         data_new['outside_sound'] = data_obj['outside_sound']
         data_new['pref_vol'] = data_obj['pref_vol']
         last_id += 1
         aug_data.append(data_new)

#print(data_array)
#print()
#print(aug_data)

{'id': 0, 'current_vol': 96.55347582937657, 'outside_sound': 'sound_0.npy', 'pref_vol': 89.73781503845449} 96.55347582937657
{'id': 1, 'current_vol': 96.55347582937657, 'outside_sound': 'sound_0.npy', 'pref_vol': 89.73781503845449} 96.55347582937657
{'id': 2, 'current_vol': 89.73781503845449, 'outside_sound': 'sound_1.npy', 'pref_vol': 109.59903228876692} 89.73781503845449
{'id': 3, 'current_vol': 89.73781503845449, 'outside_sound': 'sound_1.npy', 'pref_vol': 109.59903228876692} 89.73781503845449
{'id': 4, 'current_vol': 107.62811925310741, 'outside_sound': 'sound_2.npy', 'pref_vol': 94.54957472410238} 107.62811925310741


In [6]:
last_sound_id = int(data_array[-1]['outside_sound'].replace('sound_','').replace('.npy',''))
print(last_sound_id)

#more_aug_data = []

for data_obj in aug_data[0:10]:
    if np.random.randint(0,2) == 0:
        # add noise
        sound_data = np.load(data_path + data_obj['outside_sound'])
        noise = np.random.randn(sound_data.shape[0],sound_data.shape[1])
        map_to_small_range = interp1d([np.min(noise), np.max(noise)],[np.min(sound_data[np.nonzero(sound_data)]) / 6,np.max(sound_data[np.nonzero(sound_data)]) / 6])
        noise = [map_to_small_range(x) for x in noise]
        #sd.play(np.add(sound_data, noise))
        #sd.wait()
        last_sound_id = last_sound_id + 1
        np.save(data_path + f'sound_{last_sound_id}.npy', np.add(sound_data, noise))
        data_obj['outside_sound'] = f'sound_{last_sound_id}.npy'

with open('rec_file.txt', 'a') as f:
    for data_a in aug_data:
        str_to_save = '{' + f'\"id\": ' + str(data_a['id']) + ', \"current_vol\": '  + str(data_a['current_vol']) + ', \"outside_sound\": \"' + str(data_a['outside_sound']) +  '\", \"pref_vol\": ' + str(data_a['pref_vol']) +  '}'
        f.write(str_to_save + '\n')

2
